#### 인공신경망(ANN) 모델 살펴보기
- 구성 : 입력층 + 은닉층 + 출력층
    * 입력층 : 입력값 (<= 입력피처 수), 출력값(<=층의 퍼셉트론 수)
    * 출력층 : 입력값 (<= 이전 층의 결괏값), 출력값(<= 입력 타겟 수, 이진분류 1개, 다중분류 = 클래스 수, 회귀 => 1개)

In [116]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.random as random
import torch.nn.functional as F

random.manual_seed(12)

In [117]:
# 데이터 생성
# 피처 : 점수, 공부시간
# 타겟 : 합격/불합격

featureTS = torch.FloatTensor([[40, 4], [60, 5], [80, 6], [100, 8]])
targetTS = torch.FloatTensor([[0], [1], [1], [1]]) # 불합격 0 합격 1

print(f'[featureTS] {featureTS.shape}, {featureTS.ndim}D')
print(f'[targetTS] {targetTS.shape}, {targetTS.ndim}D')

[featureTS] torch.Size([4, 2]), 2D
[targetTS] torch.Size([4, 1]), 2D


In [118]:
# ANN 모델 
class Net(nn.Module) :
    ## 모델 구조 설정 초기화
    def __init__(self) :
        super().__init__()
        self.layer1 =nn.Linear(2, 4) # 입력층
        self.sig1 = nn.Sigmoid()

        self.layer2 =nn.Linear(4, 4) # 은닉층
        self.sig2 = nn.Sigmoid()

        self.layer3 =nn.Linear(4, 1) # 출력층
        self.sig3 = nn.Sigmoid()

    ## 순전파 함수
    def forward(self, x) :
        y1 = self.layer1(x)
        y2 = self.sig1(y1)
        y3 = self.layer2(y2)
        y4 = self.sig2(y3)
        y5 = self.layer3(y4)
        y6 = self.sig3(y5)

        return y6

In [119]:
# 모델 인스턴스 생성
model1 = Net()
model1

Net(
  (layer1): Linear(in_features=2, out_features=4, bias=True)
  (sig1): Sigmoid()
  (layer2): Linear(in_features=4, out_features=4, bias=True)
  (sig2): Sigmoid()
  (layer3): Linear(in_features=4, out_features=1, bias=True)
  (sig3): Sigmoid()
)

In [120]:
for param in model1.named_parameters() : # generater라서 for문 돌려서 출력해야함.
    print(param)

('layer1.weight', Parameter containing:
tensor([[-0.0485, -0.3779],
        [-0.0669,  0.1232],
        [-0.1292, -0.5273],
        [ 0.1941, -0.3648]], requires_grad=True))
('layer1.bias', Parameter containing:
tensor([ 0.3270,  0.3146, -0.4253,  0.2755], requires_grad=True))
('layer2.weight', Parameter containing:
tensor([[ 0.0830,  0.1318,  0.0559, -0.3738],
        [ 0.4790,  0.3443, -0.3744, -0.0544],
        [ 0.1601, -0.4446, -0.3427,  0.3137],
        [ 0.2216, -0.2283, -0.1997,  0.1099]], requires_grad=True))
('layer2.bias', Parameter containing:
tensor([ 0.0784,  0.1083, -0.0661,  0.3813], requires_grad=True))
('layer3.weight', Parameter containing:
tensor([[-0.1784, -0.2396, -0.2434, -0.3128]], requires_grad=True))
('layer3.bias', Parameter containing:
tensor([0.1423], requires_grad=True))


In [121]:
# 학습 진행
# 에포크 : 데이터를 처음부터 끝까지 한 번 학습하는 것. 
# 배치 사이즈(≒ 미니배치) : 데이터를 일정한 크기로 자른 것 ( 일반적으로는 32 --> 그냥 값이 잘 나와서..)

# 1 epoch
output = model1(featureTS)
output

tensor([[0.4043],
        [0.4043],
        [0.4043],
        [0.4043]], grad_fn=<SigmoidBackward0>)

In [122]:
# 손실 계산 = > 이진분류 -- 0.5 기준으로 판별
# 정확도(Accuracy) : TruePositives+TrueNegatives/TruePositives+TrueNegatives+FalsePositives+FalseNegatives

correct = sum((output > 0.5 ) == targetTS)
accuracy = correct / targetTS.shape[0]

print(f'accuracy => {accuracy.item()}')

# 정밀도 : TruePositives/TruePositives+FalsePositives
# 재현율 : TruePositives/TruePositives+FalseNegatives
# F1 Score : 2∗ Precision∗Recall/Precision+Recall
  

accuracy => 0.25


In [123]:
# 정밀도
correct = sum((output>0.5) == targetTS)
if not correct :
    print(f'precision : 0')

In [124]:
# 재현율
correct = sum((output>0.5) == targetTS)
correct

tensor([1])

### 손실함수 즉, 정답과 예측값 사이의 오차
- 회귀 : MSE, MAE, RMSE, ...
- 분류 : binary_cross_entropy, cross_entropy 

In [125]:
print(f'targetTS\n{targetTS}') # 정답지 
print(f'output\n{output}') # 예측값 

targetTS
tensor([[0.],
        [1.],
        [1.],
        [1.]])
output
tensor([[0.4043],
        [0.4043],
        [0.4043],
        [0.4043]], grad_fn=<SigmoidBackward0>)


In [127]:
F.binary_cross_entropy(output, targetTS)

tensor(0.8086, grad_fn=<BinaryCrossEntropyBackward0>)

In [128]:
#logit ▶ y = w1*x+w2*x+b
# binary는 sigmoid에 넣었던 값을 넣어야 함.
# cross_entropy는 X

#### 학습 진행 <hr>
- (1) 모델로 학습 진행 ==> 학습 횟수 지정
- (2) 학습결과와 정답 비교 손실 계산 ==> 손실함수
- (3) W, b 업데이트 ==> 옵티마이저

In [131]:
import torch.optim as optim

In [132]:
## 학습에 필요한 준비.

model2 = Net()

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model2.parameters())

EPOCHS = 10

In [138]:
# 학습 진행

# 학습 모드로 모델 설정
model2.train() # 배치정규화, dropout, 가중치 초기화 등의 작업을 할수 있는 환경 / 메모리 준비

for ep in range(EPOCHS) :
    ## 학습
    output=model2(featureTS) # 한 에포크 마다 새 output이 생겨야함.

    # 손실 계산
    loss = loss_fn(output, targetTS)

    # 업데이트

    optimizer.zero_grad() # 텐서의 grad 속성에 값을 초기화 --> 이전 에포크에서 들어온 값을 초기화 하려고. (값이 누적되지 않도록)

    loss.backward() #learning rate와 손실값으로 새로운 w, b 계산.

    optimizer.step() # 모델로부터 전달받은 W, b 텐서의 주소로 새로운 W, b 업데이트

    # 에포크에서 진행된 결과 출력
    
    print(f'[{ep}/{EPOCHS}] loss : {loss.item()}')

[0/10] loss : 0.647379994392395
[1/10] loss : 0.6466494798660278
[2/10] loss : 0.6459231972694397
[3/10] loss : 0.645201563835144
[4/10] loss : 0.6444849967956543
[5/10] loss : 0.6437736749649048
[6/10] loss : 0.6430677175521851
[7/10] loss : 0.6423674821853638
[8/10] loss : 0.6416731476783752
[9/10] loss : 0.6409845948219299


In [144]:
# test : モデルの性能平価　・アップデート発生してはいけない！
# モデル動作モード設定、autograd機能停止、requires_grad　tensor　停止

#테스트 모드로 모델 설정 : 배치 정규화 dropout, 가중치 초기화 기능 off
model2.eval()

with torch.no_grad():
    # 테스트
    output=model2(featureTS)

    # 손실 계산 : 테스트 데이터에 정답(타겟) 존재하는 경우 진행
    loss = loss_fn(output, targetTS)

    # 에포크
    print(f'[Test] loss : {loss.item()}')

[Test] loss : 0.6403016448020935
